## Importar el dataset de noticias en español

In [1]:
#!pip install spacy
#!python -m spacy download es
#!pip install stop_words
#!pip install dplython
#!pip install stop_words
#!pip install install --upgrade scikit-learn

In [2]:
from io import BytesIO 
import requests 
import sys
import types
import pandas as pd
import numpy as np
import spacy
from stop_words import get_stop_words
import string
import json
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n,
    sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction) 
import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split

/usr/local/src/bluemix_jupyter_bundle.v83/notebook/lib/python2.7/site-packages/scipy/sparse/lil.py:16: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/src/bluemix_jupyter_bundle.v83/notebook/lib/python2.7/site-packages/scipy/sparse/lil.py:16: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from . import _csparsetools
/usr/local/src/bluemix_jupyter_bundle.v83/notebook/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/local/src/bluemix_jupyter_bundle.v83/notebook/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from ._shortest_path impor

Conectémonos a una base de Hive, comúnmente usada en Big Data, donde tenemos almacenadas las noticias

Agregar el código del paso _"Agregar las credenciales para conectarse a la base de datos de Hive"_

In [3]:
# The code was removed by DSX for sharing.

In [4]:
from ingest.Connectors import Connectors
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

HiveloadOptions = { Connectors.Hive.HOST                        : hive_credentials["host"],
                      Connectors.Hive.PORT                      : hive_credentials["port"],
                      Connectors.Hive.DATABASE                  : hive_credentials["database"],
                      Connectors.Hive.USERNAME                  : hive_credentials["username"],
                      Connectors.Hive.PASSWORD                  : hive_credentials["password"],
                      Connectors.Hive.SOURCE_TABLE_NAME         : "noticias_sr"}

hive_df = sqlContext.read.format("com.ibm.spark.discover").options(**HiveloadOptions).load()
hive_df.printSchema()
#hive_df.show(10)

root
 |-- categoria: string (nullable = true)
 |-- url: string (nullable = true)
 |-- contenido: string (nullable = true)



Definamos una función para transformar los archivos de texto en un dataframe de Pandas

In [5]:
news_df = hive_df.toPandas()

## Preprocesemos los datos

Revisemos el estado de los datos

In [6]:
news_df.groupby('categoria').count()

,url,contenido
categoria,,
deportes,1102,1102
economia,586,586
politica,237,237
tecnologia,437,437


In [7]:
news_df.describe()

,categoria,url,contenido
count,2362,2362,2362
unique,4,1908,1859
top,deportes,https://elpais.com/deportes/2018/02/25/actuali...,
freq,1102,8,330


Como se ve en la anterior tabla, la cantidad de noticias no es unica. Quitemos las noticias que estén duplicadas o que puedan estar vacías

In [8]:
def preprocess(text_df,text_col_name):
    #Antes de quitarlos, reemplacemos los textos sin contenido por "nan"'s y después sí removemos los textos que tengan valor faltante
    text_df = text_df.apply(lambda x: x.str.strip()).replace('', np.nan)
    
    #Remover las noticias sin links
    text_df.dropna(subset=[text_col_name], inplace=True)
    
    #Remover las noticias duplicadas
    text_df.drop_duplicates(subset=[text_col_name], inplace=True)
    
    #Resetear los índices
    text_df = text_df.reset_index(drop=True)
    
    return(text_df)

In [9]:
news_df = preprocess(news_df,'contenido')
news_df.describe()

,categoria,url,contenido
count,1857,1854,1857
unique,4,1668,1857
top,deportes,https://elpais.com/tecnologia/2016/04/19/porta...,Banfield pegó y aguantó como un buen campeón d...
freq,905,6,1


Para usar _Natural Language Classifier_ de _IBM_ es necesario que ninguno de los textos supere 1024 caracteres. Para evitar esto, podemos separar las noticias en parrafos de longitud fija o en oraciones usando el paquete spaCy. 

Como el API de _NLC_ está más orientada a análisis de texto en redes sociales o en conversaciones, separemos los textos a un nivel de oraciones

### Separar oraciones

In [10]:
#Definir el idioma a usar con spaCy
nlp = spacy.load('es')

def separateSentences(text):

    #Correr spaCy en el texto
    documents = nlp(text)

    #Separar en oraciones
    sentences = [sent.string.strip() for sent in documents.sents]
    
    return sentences

/gpfs/fs01/user/s3df-546c691ce14f7e-9e6e67bd3b24/.local/lib/python2.7/site-packages/spacy/language.py:15: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tokenizer import Tokenizer
/gpfs/fs01/user/s3df-546c691ce14f7e-9e6e67bd3b24/.local/lib/python2.7/site-packages/spacy/language.py:15: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from .tokenizer import Tokenizer
/gpfs/fs01/user/s3df-546c691ce14f7e-9e6e67bd3b24/.local/lib/python2.7/site-packages/msgpack_numpy.py:179: PendingDeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpacker.unpack(stream, encoding=encoding, **kwargs)
/gpfs/fs01/user/s3df-546c691ce14f7e-9e6e67bd3b24/.local/lib/python2.7/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj

Usemos la función con un ejemplo

In [11]:
text = 'Esto, porque la que se hizo en el 2016, con todo y las críticas que ha tenido, principalmente por haber ayudado a frenar el consumo de los hogares con el incremento del IVA, sería suficiente para que el Gobierno pueda obtener los ingresos necesarios que le permitan mantener a raya el déficit fiscal. Claro está, paralelamente se tendría que organizar el gasto público e impulsar el crecimiento de la economía.'
oraciones = separateSentences(text.decode('utf-8'))
print(oraciones)

[u'Esto, porque la que se hizo en el 2016, con todo y las cr\xedticas que ha tenido, principalmente por haber ayudado a frenar el consumo de los hogares con el incremento del IVA, ser\xeda suficiente para que el Gobierno pueda obtener los ingresos necesarios que le permitan mantener a raya el d\xe9ficit fiscal.', u'Claro est\xe1, paralelamente se tendr\xeda que organizar el gasto p\xfablico e impulsar el crecimiento de la econom\xeda.']


### Remover símbolos de puntuación dentro de cada oración

Para poder pasarlo a _NLC_, es necesario remover algunos símbolos para que el servicio de _IBM_ lo pueda procesar

In [12]:
#Definir el conjunto de símbolos de puntuación a remover
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve",'¿',"¡"]

WHITES = ["", "\n", "\n\n"]

#Lista negra de simbolos
BLACKLIST = SYMBOLS + WHITES

In [13]:
def remove_punctuation(s):
    for char in BLACKLIST:
        s = s.replace(char, '')
    return(s)

Usemos la función en una de nuestras oraciones anteriores

In [14]:
remove_punctuation(oraciones[0].encode(encoding='UTF-8',errors='strict'))

'Esto porque la que se hizo en el 2016 con todo y las cr\xc3\xadticas que ha tenido principalmente por haber ayudado a frenar el consumo de los hogares con el incremento del IVA ser\xc3\xada suficiente para que el Gobierno pueda obtener los ingresos necesarios que le permitan mantener a raya el d\xc3\xa9ficit fiscal'

### Apliquemos esto a todo el texto

In [15]:
def separar_texto(text_df):
    broken_text = []
    for i in range(0,len(text_df['contenido'])):
        try:
            text = text_df['contenido'][i]
            category = text_df['categoria'][i]
            sentences = separateSentences(text)
            
            for sentence in sentences:
                sentence = remove_punctuation(sentence.encode(encoding='UTF-8',errors='strict'))
                record = (sentence, category)
                broken_text.append(record)
        except:
            continue

    broken_df = pd.DataFrame.from_records(broken_text)
    
    return(broken_df)

In [16]:
news_sentence_df = separar_texto(news_df)

Veamos como queda el dataframe después de separarlo en oraciones

In [17]:
news_sentence_df.describe()

,0,1
count,41329,41329
unique,30068,4
top,La cantante Rosalía acompañada del productor E...,deportes
freq,161,24544


Algunas oraciones son iguales, seguramente por terminaciones como <b>"ECONOMÍA Y NEGOCIOS"</b> en algunos artículos. Quitemos una vez más los duplicados

In [18]:
news_sentence_df = preprocess(news_sentence_df,0)

Algunas oraciones pueden tener más de 1024 caracteres, por lo que es mejor limitarlas en caso que existan. Empecemos por contar cuantas son

In [19]:
def count_over_limit(text_df):
    count = 0
    for txt in text_df:
        if(len(txt) >1024):
            count = count + 1
    return(count) 

count_over_limit(news_sentence_df[0])

12

Son muy pocas las noticias, por lo que podríamos quitarlas del dataset o limitarlas. Optemos por la segunda

In [20]:
def limitar_texto(text_df):
    max_char = 1024
    for i in range(0,len(text_df[0])):
        if(len(text_df.loc[i,0]) >1024):
            text_df.loc[i,0] = text_df.loc[i,0][0:1024]
        
    return(text_df)

In [21]:
news_sentence_df = limitar_texto(news_sentence_df)
count_over_limit(news_sentence_df[0])

0

### Separemos nuestros datos en datos de entrenamiento y datos de validación

In [22]:
news_train, news_test, cat_train, cat_test = train_test_split(
    news_sentence_df.drop(1, axis = 1), news_sentence_df[1], test_size=0.4, random_state=42)

#Dado el formato que requiere NLC
train = pd.concat([news_train, cat_train], axis=1)

Exportemos nuestra lista de entrenamiento a un archivo csv para pasarselo al API _NLC_

In [23]:
train.to_csv('train.csv',header=False,index=False,encoding="utf-8")

## Entrenemos el Clasificador

Conectémonos con el servicio de _Natural Language Classifier_

In [24]:
#Instalar el sdk de Watson
#!pip install --upgrade watson-developer-cloud
#Importar a Python 
from watson_developer_cloud import NaturalLanguageClassifierV1

Agregar el código del paso _Entrenar el clasificador Natural Language Classifier_

In [ ]:
#Colocar aqui Credenciales NLC

In [ ]:
# The code was removed by DSX for sharing.

In [25]:
#Definir las credenciales para poder usar el servicio
natural_language_classifier = NaturalLanguageClassifierV1(
    username=nlc_username,
    password=nlc_password
)

Enviemos los datos de entrenamiento y creemos el clasificador. Dado que toma algunas horas en entrenar, para el _workshop_ ya hemos entrenado el clasificador

In [ ]:
with open('../work/train.csv', 'rb') as training_data:
    metadata=json.dumps({'name': 'Clasificador de noticias','language': 'es'})
    classifier = natural_language_classifier.create_classifier(
        metadata=metadata,
        training_data=training_data
    )

Este es el id del clasificador que ya habíamos entrenado antes

In [26]:
#Guardemos el id del clasificador para revisar su estado y pedirle clasificaciones
classifier_id = 'f7ea68x308-nlc-144'

Revisemos el estado del clasificador con el id que retorna el API. Las opciones son _training_ o _available_. En este caso ya está disponible

In [27]:
status = natural_language_classifier.get_classifier(classifier_id)
print (json.dumps(status, indent=2))

{
  "status": "Available", 
  "name": "Clasificador de noticias", 
  "language": "es", 
  "created": "2018-03-06T21:19:20.203Z", 
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f7ea68x308-nlc-144", 
  "status_description": "The classifier instance is now available and is ready to take classifier requests.", 
  "classifier_id": "f7ea68x308-nlc-144"
}


{
  "status": "Training", 
  "name": "Clasificador de noticias", 
  "language": "es", 
  "created": "2018-03-06T02:38:16.160Z", 
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f7e8dex307-nlc-30", 
  "status_description": "The classifier instance is in its training phase, not yet ready to accept classify requests", 
  "classifier_id": "f7e8dex307-nlc-30"
}

#### Una vez ya esté entrenado, revisemos que tan bueno es clasificando noticias que estaban en los datos de entrenamiento

In [28]:
#Prueba 1
texto_1 = 'En pleno debate electoral, cuando los candidatos a la presidencia se la juegan apostando por subir o no subir los impuestos, lo que a los técnicos les parece populismo, el propio Fondo Monetario Internacional (FMI), que por demás, sorprendió con una baja en su proyección de crecimiento de la economía en el 2018 (de 3 a 2,7 %) y recomendó seguir recortando las tasas de interés del Emisor, también le dijo al ministro de Hacienda, Mauricio Cárdenas, que no ve la necesidad de una nueva reforma tributaria en el país.'
prueba_1 = natural_language_classifier.classify(classifier_id, texto_1)
print(json.dumps(prueba_1, indent=2))

{
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f7ea68x308-nlc-144", 
  "text": "En pleno debate electoral, cuando los candidatos a la presidencia se la juegan apostando por subir o no subir los impuestos, lo que a los t\u00e9cnicos les parece populismo, el propio Fondo Monetario Internacional (FMI), que por dem\u00e1s, sorprendi\u00f3 con una baja en su proyecci\u00f3n de crecimiento de la econom\u00eda en el 2018 (de 3 a 2,7 %) y recomend\u00f3 seguir recortando las tasas de inter\u00e9s del Emisor, tambi\u00e9n le dijo al ministro de Hacienda, Mauricio C\u00e1rdenas, que no ve la necesidad de una nueva reforma tributaria en el pa\u00eds.", 
  "classes": [
    {
      "class_name": "economia", 
      "confidence": 0.9938446210868216
    }, 
    {
      "class_name": "politica", 
      "confidence": 0.0047897735231073445
    }, 
    {
      "class_name": "deportes", 
      "confidence": 0.0013656053900711037
    }
  ], 
  "classifier_id": 

In [29]:
#Prueba 2
texto_2 = 'Aunque la ventaja es para el Real Madrid, tanto este equipo como el Paris Saint-Germain, que llega 3-1 abajo en la serie de octavos de final, llegan al Parque de los Príncipes a jugarse la temporada, nada menos. Más que el paso a la siguiente fase, no solo se juegan su presente, sino también su futuro. A las 2:45 de la tarde comienza a rodar la bola. Transmite ESPN 2.'
classes = natural_language_classifier.classify(classifier_id, texto_2)
print(json.dumps(classes, indent=2))

{
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f7ea68x308-nlc-144", 
  "text": "Aunque la ventaja es para el Real Madrid, tanto este equipo como el Paris Saint-Germain, que llega 3-1 abajo en la serie de octavos de final, llegan al Parque de los Pr\u00edncipes a jugarse la temporada, nada menos. M\u00e1s que el paso a la siguiente fase, no solo se juegan su presente, sino tambi\u00e9n su futuro. A las 2:45 de la tarde comienza a rodar la bola. Transmite ESPN 2.", 
  "classes": [
    {
      "class_name": "deportes", 
      "confidence": 0.9961117090763454
    }, 
    {
      "class_name": "politica", 
      "confidence": 0.0024911271985377436
    }, 
    {
      "class_name": "economia", 
      "confidence": 0.001397163725116979
    }
  ], 
  "classifier_id": "f7ea68x308-nlc-144", 
  "top_class": "deportes"
}


#### Veamos cuál fue el desempeño del modelo sobre los datos de _test_

Lo primero es obtener una predicción para cada una de las instancias que guardamos en nuestro test set

In [30]:
news_test = news_test.reset_index(drop= True)

In [31]:
cat_predicted = []
for i in range(0,len(news_test)):
    #Extract the text to use in the API call
    text = news_test.loc[i,0]
    
    #Get the result for the classification
    class_result = natural_language_classifier.classify(classifier_id, text)
    predicted_class_i = class_result['top_class']
    cat_predicted.append(predicted_class_i)

Con esto ya podemos evaluar que tan preciso es nuestro modelo de evaluar noticias a nivel de oraciones

In [35]:
accuracy_score(cat_test, cat_predicted)

0.7882470451140337

Veamos específicamente que tanto se equivoca en cada clase de categoría

In [36]:
confusion_matrix(cat_test, cat_predicted, labels=['economia','deportes','politica', 'tecnologia'])

array([[2513,  164,  100,    0],
       [ 107, 6173,   29,    0],
       [ 169,   96,  784,    0],
       [1353,  457,   69,    0]])

## Juguemos con el clasificador

In [37]:
#Prueba 2
texto_2 = 'El presidente Florentino Pérez manifestó que no se dejará presionar por la oposición al interior del Madrid'
classes = natural_language_classifier.classify(classifier_id, texto_2)
print(json.dumps(classes, indent=2))

{
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f7ea68x308-nlc-144", 
  "text": "El presidente Florentino P\u00e9rez manifest\u00f3 que no se dejar\u00e1 presionar por la oposici\u00f3n al interior del Madrid", 
  "classes": [
    {
      "class_name": "politica", 
      "confidence": 0.8393619964696493
    }, 
    {
      "class_name": "deportes", 
      "confidence": 0.14918065313906154
    }, 
    {
      "class_name": "economia", 
      "confidence": 0.01145735039128919
    }
  ], 
  "classifier_id": "f7ea68x308-nlc-144", 
  "top_class": "politica"
}
